In [1]:
import numpy as np
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

import pandas as pd

from IPython.display import Image
from IPython.core.display import HTML 

PATH = "/home/jma/Desktop/march_embedding/"

import keras
from keras import *
from keras.layers import Embedding, Input,concatenate, Flatten,dot, Dense,LSTM
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

/home/jma/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/jma/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<h2>I want to look at how a batch of reviews look in and out of Embedding layer</h2>

In [2]:
#load reviews and limit to 10
train=pd.read_csv('/media/jma/DATA/amazon_review_full_csv/train.csv',header=0,nrows=10, names=['rating','short','fullreview'])
train['fullreview']=train['fullreview'].replace(regex=True,to_replace='[^\sa-zA-Z]', value=r'')
X=train.fullreview.str.lower()
X=X.str.split() #now each instance is a list of tokens 


#loop through each row and create a dictionary of words


dct_word_freq={}

for x in X.iteritems():
    for v in x[1]:
        if v in dct_word_freq:
            dct_word_freq[v]+=1
        else:
            dct_word_freq[v]=1
            

#now, could remove those with low number of instances (or set them to UNK)
#TODO



#now create a dictionary where each unique word is mapped to an int (add 2 so we dont have 0 (padding) or 1 (unk) used)
dct_word_to_int={o:i+2 for i,o in enumerate(set(dct_word_freq.keys()))}
dct_word_to_int['unk']=1

print(dct_word_to_int['the'])#mapped to 71
print(dct_word_to_int['unk'])#mapped to 1


#now replace each word in the list of tokens in X with its corresponding int
#would so same exact thing for test sets

def replace_to_int(x,dct_word_to_int):
    hold_ints=[]
    for token in x:
        if token in dct_word_to_int:
            hold_ints.append(dct_word_to_int[token])
        else:
            hold_ints.append(1) #unknown
    return(hold_ints)
    
z=X.apply(replace_to_int,args=(dct_word_to_int,))
z.head()

22
1


0    [223, 136, 15, 256, 14, 65, 318, 82, 5, 182, 3...
1    [164, 359, 15, 256, 14, 6, 77, 157, 82, 57, 22...
2    [22, 175, 14, 151, 218, 57, 315, 73, 341, 271,...
3    [261, 22, 72, 129, 258, 219, 248, 234, 45, 28,...
4    [181, 15, 103, 82, 5, 57, 272, 247, 97, 22, 35...
Name: fullreview, dtype: object

In [3]:
#now can use keras padding

maxlen=50

#shape is now (batche size, # time steps)
z=keras.preprocessing.sequence.pad_sequences(z.values,maxlen=maxlen)
z.shape

(10, 50)

In [7]:
vocab_size=len(dct_word_to_int)
embedding_size=10

embedding_layer = Embedding(output_dim=embedding_size, input_dim=vocab_size,input_length=maxlen, mask_zero=True)

x = Input(shape=[maxlen])
embedding = embedding_layer(x)
out=LSTM(128, dropout=0.2, recurrent_dropout=0.2)(embedding)

model = Model(inputs=x, outputs=out)

#so out (and thus into LSTM) of the embedding is size (batch size, # time steps (aka sequence length), embedding size  )
#size out of the LSTM is (batches and dimension of the LSTM)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 50, 10)            3600      
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               71168     
Total params: 74,768
Trainable params: 74,768
Non-trainable params: 0
_________________________________________________________________
